In [608]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import math

In [609]:
p_a=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 absorption.csv')
p_f=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 fission.csv')
p_s=pd.read_csv('/home/shochcho/Documents/Thermal_hydraulics_and_neutronics_coupling-main/cross section data/Pu-239 scattering.csv')

atom_density=6.02*10**23*19.84/239


In [610]:
x=10**-6*np.array(p_f['Incident energy'])
y=10**-24*atom_density*np.array(p_f['Cross section'])
pf=interp1d(x,y)

x=10**-6*np.array(p_a['Incident energy'])
y=10**-24*atom_density*np.array(p_a['Cross section'])
pa=interp1d(x,y)

x=10**-6*np.array(p_s['Incident energy'])
y=10**-24*atom_density*np.array(p_s['Cross section'])
ps=interp1d(x,y)

In [611]:
def absorbtion_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pa(neutron_energy)
	else:
		return 0
	


def scattering_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return ps(neutron_energy)
	else:
		return 0

def fission_cross_section(geometry,neutron_energy):
	neutron_energy=neutron_energy
	if geometry=='p':
		return pf(neutron_energy)
	else:
		return 0
		
def energy_updater(energy,molar_mass):
	A=molar_mass
	E=energy
	alpha=1-((A-1)/(A+1))**2
	delta_E=np.random.uniform(0,alpha*E)
	
	return E-delta_E

def watt_spectrum(energy):
    
    prob=0.4865*np.sinh(np.sqrt(2*energy))*np.exp(-energy) 
    
    return prob

In [612]:
skip=20
number_of_neutrons=1000
no_gen_sim=100				      
position_matrix=np.zeros((number_of_neutrons,3))
k_eff=np.zeros(no_gen_sim)
energy=[]
new_gen_neutron=np.zeros(no_gen_sim)
weight0=np.ones(number_of_neutrons)
weight1=np.ones(number_of_neutrons)

for i in range (number_of_neutrons):
	position_matrix[i][0]=np.random.uniform(0,6.3849)                    #radius record 
	position_matrix[i][1]=np.random.uniform(-np.pi,np.pi)                #theta record 
	position_matrix[i][2]=np.random.uniform(-np.pi/2,np.pi/2)	  		 #phi record 
	energy.append(np.random.uniform(2,4))

r=np.ones(number_of_neutrons)*position_matrix[i][0]
theta=np.ones(number_of_neutrons)*position_matrix[i][1]
phi=np.ones(number_of_neutrons)*position_matrix[i][2]
energy_new=np.ones(number_of_neutrons)*energy

k_eff=[]



In [613]:
for j in range(no_gen_sim):
		
	for i in range (number_of_neutrons):
        
        #location copy
		x=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.cos(position_matrix[i][1])
		y=position_matrix[i][0]*math.sin(position_matrix[i][2])*math.sin(position_matrix[i][1])
		z=position_matrix[i][0]*math.cos(position_matrix[i][2])
		Energy = energy[i]
		alive=True
		
		while alive:
		
		# core 
			if math.sqrt(x**2+y**2+z**2)<6.3849:
				#that means the neutron is inside the core 
				
				sigma_s=scattering_cross_section("p",Energy)
				sigma_a=absorbtion_cross_section("p",Energy)
				sigma_f=fission_cross_section("p",Energy)
				
				total_sigma=sigma_s+sigma_f+sigma_a
				random_parameter=np.random.rand()
				theta_move=np.random.uniform(0,2*np.pi)
				phi_move=np.random.uniform(0,np.pi)
					
				l= -math.log(1-random_parameter)/total_sigma
					
				x = x+l*math.sin(phi_move)*math.cos(theta_move)
				y = y+l*math.sin(phi_move)*math.sin(theta_move)
				z = z+l*math.cos(phi_move)
				
				reaction_type=np.random.choice(["fission","scattering","absorbtion"],p=[sigma_f/total_sigma,sigma_s/total_sigma,sigma_a/total_sigma])
				
				#need to figure out how to make a bias here ------done
				
				if reaction_type =="scattering":
					
					Energy =energy_updater(Energy,239)

					if Energy<0.01:
						Energy=2
						
					weight1[i]=weight0[i]*(sigma_s/total_sigma)*weight1[i]
			    		
				elif reaction_type=="fission":
					
					neu=2.7
					meu=np.random.rand()
					if meu<(neu-int(meu)):
						new_neutron=int(neu)
					else: 
						new_neutron=int(neu)+1

					new_gen_neutron[j]=new_gen_neutron[j]+new_neutron

					weight1[i]=new_neutron*weight0[i]*(1+sigma_f/total_sigma)
					
					#location has to be saved for new initiation 
					#there will be a loop to create the same location in a matrix
					#for the same number of neutron 
					
					for k in range(1):
					
				
						r[i]=(math.sqrt(x**2+y**2+z**2))
						theta[i]=(math.atan(y/x))
						phi[i]=(math.atan(math.sqrt(x**2+y**2)/z)) 

						""" need watt spectrum rejection sampling """
						fish_flag=True
						while fish_flag:
							fiss_energy=np.random.uniform(0,12)
							fiss_prob=watt_spectrum(fiss_energy)
							jeeta=np.random.rand()

							if jeeta>fiss_prob:
								energy_new[i]=fiss_energy
								fish_flag=False
			
					
					 
				else:
					#russian rulete
					if weight0[i]<0.02:
						meu=np.random.rand()
						d=np.random.randint(2,10)
						if meu<=1/d:
							weight0[i]=d*weight0[i]
						else:
							alive=False
					else:
						
						alive=False
				
					weight1[i]=weight0[i]*(1-sigma_a/total_sigma)*weight1[i] 
			else:
				
				alive=False

	position_matrix=np.zeros((len(r),3))
	for k in range(len(r)):

		position_matrix[k][0]=r[k]
		position_matrix[k][1]=theta[k]
		position_matrix[k][2]=phi[k]

		
	energy=np.zeros(len(energy_new))
	energy=energy_new
	k_eff.append(np.sum(weight1)/np.sum(weight0))
 
	if j<9 :
		if j<=skip:
			print('batch',"  ",j+1,"->1",'%.6f' % k_eff[j])
		else: 
			b=np.mean(k_eff[skip:])
			a=np.std(k_eff[skip:])
			print('batch',"  ",j+1,"->1",'%.6f' % k_eff[j],'   ','%.6f' % b,"-\+",'%.6f' %a)
	elif j<99:
		if j<=skip:
			print('batch'," ",j+1,"->1",'%.6f' % k_eff[j])
		else: 
			b=np.mean(k_eff[skip:])
			a=np.std(k_eff[skip:])
			print('batch'," ",j+1,"->1",'%.6f' % k_eff[j],'   ','%.6f' % b ,"-\+",'%.6f' %a)
	else:
		if j<=skip:
			print('batch',j+1,"->1",'%.6f' % k_eff[j])
		else: 
			b=np.mean(k_eff[skip:])
			a=np.std(k_eff[skip:])
			print('batch','',j+1,"->1",'%.6f' % k_eff[j],'   ','%.6f' %b,"-\+",'%.6f' %a)



batch    1 ->1 1.435794
batch    2 ->1 1.767511
batch    3 ->1 2.132448
batch    4 ->1 2.328262
batch    5 ->1 2.449847
batch    6 ->1 2.548455
batch    7 ->1 2.595847
batch    8 ->1 2.648139
batch    9 ->1 2.665672
batch   10 ->1 2.675200
batch   11 ->1 2.685967
batch   12 ->1 2.693076
batch   13 ->1 2.694484
batch   14 ->1 2.696476
batch   15 ->1 2.698603
batch   16 ->1 2.698160
batch   17 ->1 2.699284
batch   18 ->1 2.700504
batch   19 ->1 2.699243
batch   20 ->1 2.701135
batch   21 ->1 2.700993
batch   22 ->1 2.700971     2.700982 -\+ 0.000011
batch   23 ->1 2.700945     2.700970 -\+ 0.000020
batch   24 ->1 2.703520     2.701607 -\+ 0.001104
batch   25 ->1 2.703520     2.701990 -\+ 0.001249
batch   26 ->1 2.703520     2.702245 -\+ 0.001275
batch   27 ->1 2.703520     2.702427 -\+ 0.001262
batch   28 ->1 2.703520     2.702564 -\+ 0.001235
batch   29 ->1 2.703520     2.702670 -\+ 0.001202
batch   30 ->1 2.703520     2.702755 -\+ 0.001169
batch   31 ->1 2.703520     2.702825 -\+ 0.001

In [614]:
print(np.mean(k_eff[skip:]),"-\+",np.std(k_eff[skip:]))

2.703424442214061 -\+ 0.0004845354593504819
